In [37]:
import pandas as pd 
import numpy as np

df = pd.read_table('spambase.data', header=None, delimiter=',')
df = df.sample(frac=1)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
12,0.00,0.69,0.34,0.0,0.34,0.00,0.0,0.00,0.00,0.00,...,0.000,0.056,0.000,0.786,0.000,0.0,3.728,61,261,1
3927,0.07,0.03,0.18,0.0,0.10,0.03,0.0,0.00,0.40,0.00,...,0.188,0.148,0.035,0.000,0.010,0.0,3.233,66,1387,0
3352,0.00,0.00,0.08,0.0,0.00,0.16,0.0,0.08,0.08,0.00,...,0.069,0.103,0.000,0.000,0.011,0.0,2.440,18,598,0
1278,0.00,0.89,1.15,0.0,0.12,0.00,0.0,0.12,0.25,0.12,...,0.000,0.185,0.000,0.370,0.061,0.0,2.878,84,475,1
3535,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.185,0.743,0.000,0.000,0.000,0.0,4.476,14,94,0


In [38]:
split = len(df) * 2 // 3

X_tr = df.iloc[:split, :-1].to_numpy()
X_v = df.iloc[split:, :-1].to_numpy()

Y_tr = df.iloc[:split, -1].to_numpy()
Y_v = df.iloc[split:, -1].to_numpy()

X_tr.shape, X_v.shape, Y_tr.shape, Y_v.shape

((3067, 57), (1534, 57), (3067,), (1534,))

In [39]:
ones = np.ones((X_tr.shape[0], 1))
diagY_tr = np.diag(Y_tr)
a = np.linalg.pinv(diagY_tr @ X_tr @ X_tr.T @ diagY_tr) @ ones
a.shape

(3067, 1)

In [40]:
W = X_tr.T @ diagY_tr @ a
W.shape

(57, 1)

In [41]:
Yhat_tr = np.sign(X_tr @ W)
Yhat_v = np.sign(X_v @ W)
Yhat_tr.shape, Yhat_v.shape

((3067, 1), (1534, 1))

In [ ]:
smape = lambda Y, Yhat: np.mean(np.abs(Y - Yhat) / (np.abs(Y) + np.abs(Yhat)))

rmse = lambda Y, Yhat: np.sqrt(np.mean((Y - Yhat)**2))

accuracy = lambda Yhat, Y: np.mean(np.sign(Yhat) == Y)

precision = lambda Yhat, Y: TP(Yhat, Y) / (TP(Yhat, Y) + FP(Yhat, Y)) if (TP(Yhat, Y) + FP(Yhat, Y)) != 0 else 0

recall = lambda Yhat, Y: TP(Yhat, Y) / (TP(Yhat, Y) + FN(Yhat, Y)) if (TP(Yhat, Y) + FN(Yhat, Y)) != 0 else 0

f1_score = lambda Yhat, Y: (2 * precision(Yhat, Y) * recall(Yhat, Y)) \
    / (precision(Yhat, Y) + recall(Yhat, Y)) if (precision(Yhat, Y) + recall(Yhat, Y)) != 0 else 0

# Compute confusion matrix for precision, recall, and harmonic mean
TP = lambda Yhat, Y: np.sum((Yhat == 1) & (Y == 1))  # True Positives
FP = lambda Yhat, Y: np.sum((Yhat == 1) & (Y != 1))  # False Positives
FN = lambda Yhat, Y: np.sum((Yhat != 1) & (Y == 1))  # False Negatives

# Print ERROR%
print(f'Training Accuracy (%): {accuracy(Yhat_tr, Y_tr) * 100:.2f}')
print(f'Validating Accuracy (%): {accuracy(Yhat_v, Y_v) * 100:.2f}')
print()
print(f'Training F Measure (%): {f1_score(Yhat_tr, Y_tr) * 100:.2f}')
print(f'Validating F Measure (%): {f1_score(Yhat_v, Y_v) * 100:.2f}')

Training SMAPE (%): 60.94
Validating SMAPE (%): 62.24

Training RMSE (%): 79.62
Validating RMSE (%): 80.22

Training Accuracy (%): 39.06
Validating Accuracy (%): 37.76

Training F Measure (%): 56.68
Validating F Measure (%): 55.27
